Analyzing Covid-19 Dataset Obtained from Webhose.io 

#Group 1 = Corliss Spencer + Yulia Trask + Nata Berishvili + Chris Kremer + Jason Galvan

In [26]:
pwd

'/Users/batman/Downloads'

In [49]:
pip install python-decouple

Note: you may need to restart the kernel to use updated packages.


In [34]:
import nltk
import numpy as np
import pandas as pd
import statistics
import csv
from operator import itemgetter
import matplotlib.pyplot as plt
from decouple import config
import requests
from pprint import pprint

In [27]:
#Importing Webhose json file --obtained from teammate Corliss Spencer  

import json

In [30]:
#Calling the Webhose json file, saving to variable newsfeeds_read, and checking the length of the data
json_data=open('/Users/batman/Downloads/webhose_Covid_Virus.json').readlines()
newsfeeds_read = []
for line in json_data:
    newsfeeds_read.append(json.loads(line))
print(len(newsfeeds_read))

43900


In [35]:
#Converting the Webhose data into Pandas DataFrame
df1 = pd.DataFrame.from_dict(newsfeeds_read)

In [39]:
print(df1.columns)

Index(['thread', 'uuid', 'url', 'ord_in_thread', 'parent_url', 'author',
       'published', 'title', 'text', 'highlightText', 'highlightTitle',
       'highlightThreadTitle', 'language', 'external_links', 'external_images',
       'entities', 'rating', 'crawled', 'updated'],
      dtype='object')


In [40]:
#Cheking the number of rows and columns in our Newsfeeds data: 43900 rows and 19 columns
df1.shape

(43900, 19)

#Deduplication of the Webhose Dataset: Simhash Algorithm   

Using SimHash with a small distance provides sufficient accuracy.  Therefore we will not be using Word2Vec.  

In [41]:
pip install simhash

Note: you may need to restart the kernel to use updated packages.


In [42]:
import webhoseio, os
from gensim.models import KeyedVectors
import json
from simhash import Simhash, SimhashIndex
import numpy as np

In [43]:
feeds = []
i = 0
for feed in newsfeeds_read:
    feed['id'] = i
#    print(feed['id'], str(feed['title']))
#  Remove comment to see all the titles
    i += 1
    feeds.append(feed)

In [44]:
#The data set includes ids, titles, text, and urls 
feeds_original= []

for feed in newsfeeds_read:
    feeds_original.append((feed['id'], feed['title'], feed['text'], feed['url']))

In [46]:
#Simhas Algorithm to Remove Duplicates 

In [47]:
import logging
logging.getLogger('simhash').setLevel(logging.CRITICAL)

distance = 6
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds]
index = SimhashIndex(objs, k=distance)

In [48]:
dup_indices = []
dict1= dict()

for id in range(len(feeds)):
    dict1[id] = [id]

for k in range(len(feeds)):
    feed_sel = feeds[k]
    feed_hash = Simhash(str(feed_sel['title']))
    indices = index.get_near_dups(feed_hash)

    for i in indices:
        if (i not in dup_indices) and (i != k) and (k not in dict1[int(i)]):
            dup_indices.append(i)
        dict1[i] = dict1[int(i)].append(int(k))
   

In [49]:
#dup_indices contains all duplicates ~ which we will subtract from our Webhose Dataset 

In [50]:
print(len(dup_indices))

18845


In [51]:
#Printing the length of our original Webhose Dataset 
print(len(feeds))

43900


In [52]:
#Check the size of duplicates of the titles. The size of dup_titles matches the size of dup_indices: 18845
dup_titles = []

for dupi in dup_indices:
    dup_titles.append(feeds[int(dupi)]['title'])
print(len(dup_titles))   

18845


In [53]:
#Duplicates of ids, titles, text, and url
feeds_dups =[]

for dupi in dup_indices:
    feeds_dups.append((feeds[int(dupi)]['id'], feeds[int(dupi)]['title'], feeds[int(dupi)]['text'], feeds[int(dupi)]['url']))
    #print(feeds[int(dupi)]['id'], feeds[int(dupi)]['title'], feeds[int(dupi)]['text'])

In [54]:
#Check the size of duplicates. It's the same size of dup_indices and duplicates of titles.
print(len(feeds_dups))

18845


In [55]:
#Deduplicated dataset: ids, titles, text, and url ~ 25055 is the proper length 

clean_data=set(feeds_original)-set(feeds_dups)
print(len(clean_data))

25055


In [56]:
#2nd Method: Deduplicated dataset: ids, titles, and text
deduplicated_data = (set(feeds_original).difference(feeds_dups))
print(len(deduplicated_data))

25055


In [57]:
#Sort the Deduplicated dataset: ids, titles, and text
dedupdata_sorted = sorted(clean_data, reverse=False)

In [58]:
#ensuring correct lenth 
len(dedupdata_sorted)

25055

In [59]:
#Storing the Deduplicated dataset and Sorted dataset (ids, titles, and text) to json files:
import json

with open('/Users/batman/Downloads/webhose_Covid_dedup.json', "w") as data_file:
    for x in dedupdata_sorted:
        line = json.dumps(x)
        data_file.write(line)
        data_file.write("\n")

In [60]:
#Reading JSON file into Python array of JSON objects and printing the size of the deduplicated data ~ ensuring proper length 25055

import json

dedup=open('/Users/batman/Downloads/webhose_Covid_dedup.json').readlines()
dedup_feeds = []
for line in dedup:
    dedup_feeds.append(json.loads(line))
print(len(dedup_feeds))

25055


#Topic Clustering:  Comparing topics within Fake News dataset obtained via NewsGuardTech.com to our Deduplicated Webhose Dataset

Use of Gensim LDA Modeling (train (Webhose) / (test(Fake News)  
   

In [1]:
!pip install iexfinance
!pip install pandas
!pip install matplotlib
!pip install gensim
!pip install pyLDAvis
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import requests, json, re
import pandas as pd
import matplotlib
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from datetime import datetime
from iexfinance.stocks import get_historical_data

stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 

def text_cleanup(input_text):
    one = " ".join([i for i in input_text.lower().split() if i not in stopwords])
    two = "".join(re.sub(r'[^a-zA-Z ]', '', i) for i in one if i not in punctuation)
    three = [WordNetLemmatizer().lemmatize(i) for i in two.split()]
    return three

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

In [6]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [25]:
#LDA Model Built Upon Deduplicated Webhose dataset 

In [62]:
#Importing Webhose Deduplicated Dataset 
json_data=open('/Users/batman/Downloads/webhose_Covid_dedup (1).json').readlines()
dedup_subset = []
for line in json_data:
    dedup_subset.append(json.loads(line))
print(len(dedup_subset))

25055


In [63]:
#Converting json file to Pandas Dataframe
df6 = pd.DataFrame(dedup_subset, columns=['id', 'title', 'text', 'url'])

In [64]:
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim
import os

text = df6[['text']].applymap(text_cleanup)['text']
dictionary = Dictionary(text)
dictionary.filter_extremes(no_below=10, no_above=0.8)
corpora = [dictionary.doc2bow(doc) for doc in text]

# Running and Trainign LDA model on the document term matrix.
lda_model = ldamodel.LdaModel(corpora, num_topics=6, id2word = dictionary, passes=50)

In [65]:
# Printing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpora]

[(0,
  '0.012*"it" + 0.009*"said" + 0.008*"like" + 0.008*"people" + 0.008*"time" + '
  '0.008*"one" + 0.007*"get" + 0.006*"going" + 0.006*"say" + 0.005*"make"'),
 (1,
  '0.010*"said" + 0.007*"u" + 0.006*"government" + 0.006*"trump" + '
  '0.006*"country" + 0.006*"state" + 0.006*"coronavirus" + 0.005*"vaccine" + '
  '0.005*"people" + 0.004*"also"'),
 (2,
  '0.016*"said" + 0.013*"school" + 0.012*"case" + 0.012*"health" + '
  '0.012*"state" + 0.010*"county" + 0.008*"mask" + 0.008*"student" + '
  '0.007*"new" + 0.006*"public"'),
 (3,
  '0.007*"season" + 0.007*"year" + 0.006*"game" + 0.006*"team" + 0.006*"july" '
  '+ 0.005*"event" + 0.005*"sport" + 0.005*"new" + 0.005*"player" + '
  '0.005*"first"'),
 (4,
  '0.061*"market" + 0.043*"chart" + 0.027*"size" + 0.027*"price" + '
  '0.025*"sale" + 0.023*"product" + 0.023*"unit" + 0.022*"segmentation" + '
  '0.021*"global" + 0.019*"million"'),
 (5,
  '0.010*"company" + 0.007*"business" + 0.007*"million" + 0.007*"year" + '
  '0.005*"service" + 0.00

In [66]:
# Compute Perplexity Score = Lower is Better
print('\nPerplexity Score: ', lda_model.log_perplexity(corpora))

# Compute Coherence Score = Higer is Better
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=text, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity Score:  -7.745679062973435

Coherence Score:  0.565664778267226


In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpora, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.173217 -0.098421       1        1  33.524158
5     -0.044180  0.045386       2        1  28.078344
1      0.035463 -0.025166       3        1  16.703062
0      0.146180 -0.130470       4        1  16.346743
3      0.040761  0.130364       5        1   3.627262
2     -0.005007  0.078307       6        1   1.720435, topic_info=        Term        Freq       Total Category  logprob  loglift
370     mask  225.000000  225.000000  Default  30.0000  30.0000
43   vaccine  190.000000  190.000000  Default  29.0000  29.0000
114      new  128.000000  128.000000  Default  28.0000  28.0000
188       dr   72.000000   72.000000  Default  27.0000  27.0000
118  patient   76.000000   76.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
126     rate    2.014677   41.433449   Topic6  -5.0220   1.0390
225     need    2.130530   88.761169   Topic6  -4.9661   0.3330
66     could    2.063468   72.457207   Topic6  -4.9981   0.5040
304    state    2.032731   75.180305   Topic6  -5.0131   0.4521
187  disease    2.069431  110.573181   Topic6  -4.9952   0.0842

[336 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
281       1  0.272420  action
281       2  0.435872  action
281       3  0.108968  action
281       5  0.163452  action
316       1  0.303200     ago
...     ...       ...     ...
159       5  0.084622    year
267       2  0.154661    york
267       3  0.231991    york
267       4  0.115995    york
267       6  0.425316    york

[895 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 2, 1, 4, 3])

In [68]:
#Importing Fake News Dataset in Order to Test via the Trained LDA Model 

In [69]:
dfFake = pd.read_csv('/Users/batman/Desktop/fake_news.csv')

In [70]:
dfFake2=dfFake.head(72)

In [71]:
dfFake3 = pd.DataFrame(dfFake2, columns=['id','text'])

In [72]:
#Filling NaNs Column with Dummy Data 
dfFake4 =dfFake3.fillna("Hello")

In [73]:
dfFake4.head(3)

,id,text
0,Hello,In this report we take an inside look at Event...
1,Hello,News and articles about CORONAVIRUS are everyw...
2,Hello,"First of all, Wuhan is a place and not a race,..."


In [74]:
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim
import os

text = dfFake4[['text']].applymap(text_cleanup)['text']
dictionary2 = Dictionary(text)
dictionary2.filter_extremes(no_below=10, no_above=0.8)
corpora2 = [dictionary.doc2bow(doc) for doc in text]

In [75]:
#Analyzing Fake News Dataset to Topics Identified via LDA Model (Trained on Webhose Dataset) 

In [78]:
Test2=[]
i= 0
x= 0
while x < 72:
    web1 = corpora2[i]
    vector = lda_model[web1]# Obtaining Topic Probability Distribution for each Document & Printing Below (topic#, probability#) 
    print(vector) 
    x= x+1
    i= i +1
    Test2.append(vector)

[(0, 0.2528181), (1, 0.5286063), (2, 0.016974408), (3, 0.08342626), (5, 0.11688795)]
[(0, 0.20512003), (1, 0.72005534), (2, 0.05937423), (3, 0.014680187)]
[(0, 0.15523767), (1, 0.7525919), (2, 0.04392438), (4, 0.013870515), (5, 0.03412483)]
[(0, 0.3399424), (1, 0.3525031), (2, 0.29703745)]
[(0, 0.13714291), (1, 0.54696655), (2, 0.23838954), (4, 0.031402), (5, 0.045596965)]
[(0, 0.07140684), (1, 0.6786094), (2, 0.2326331)]
[(0, 0.23453662), (1, 0.32431683), (2, 0.37435904), (4, 0.013665704), (5, 0.05019265)]
[(0, 0.29805142), (1, 0.39490536), (2, 0.19060513), (3, 0.113052584)]
[(0, 0.17360628), (1, 0.7109657), (2, 0.044766694), (5, 0.06421615)]
[(0, 0.22808452), (1, 0.44206408), (2, 0.30012167), (5, 0.029175486)]
[(1, 0.601943), (2, 0.387504)]
[(0, 0.14736468), (1, 0.45202425), (2, 0.26393944), (3, 0.12119851), (5, 0.014504907)]
[(0, 0.45613), (1, 0.2740025), (2, 0.23858577), (4, 0.014761297), (5, 0.016220627)]
[(0, 0.22946326), (1, 0.7252406), (2, 0.04333412)]
[(0, 0.28609493), (1, 0.6

#Using Gensim Sparse Vector + TF-IDF Model to Identify similarity between Webhose Dataset and Fake News Dataset 

Similarity defined by vectoral analysis of text:  if Webhose Article similarity score is >= .65 to Fake News text, then Webhose Article defined as fake news and saved as "Webhose Fake"

Two datasets will be produced:  Webhose Real vs. Webhose Fake 

In [21]:
!pip install iexfinance
!pip install pandas
!pip install matplotlib
!pip install gensim
!pip install pyLDAvis
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [22]:
#Importing Libraries to Clean + Tokenize the Dataset for Processing 

In [23]:
import requests, json, re
import pandas as pd
import matplotlib
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from datetime import datetime
from iexfinance.stocks import get_historical_data

stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 

def text_cleanup(input_text):
    one = " ".join([i for i in input_text.lower().split() if i not in stopwords])
    two = "".join(re.sub(r'[^a-zA-Z ]', '', i) for i in one if i not in punctuation)
    three = [WordNetLemmatizer().lemmatize(i) for i in two.split()]
    return three

In [24]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

In [25]:
df2 = pd.read_csv('/Users/batman/Desktop/fake_news.csv')

In [26]:
df2.head(2)

,ur1,title,url2,text
0,ActivistPost.com,Did Bill Gates and world economic forum predic...,https://www.nexusnewsfeed.com/article/human-ri...,In this report we take an inside look at Event...
1,AgeOfAutism.com,Coronavirus Can Be Caused By Viral Interferenc...,https://www.ageofautism.com/2020/01/coronaviru...,News and articles about CORONAVIRUS are everyw...


In [27]:
df3=df2.head(72)

In [28]:
df4 = pd.DataFrame(df3, columns=['id','text'])

In [29]:
df4.head(2)

,id,text
0,NaN,In this report we take an inside look at Event...
1,NaN,News and articles about CORONAVIRUS are everyw...


In [30]:
#Need to Insert Dummy Data for NaN for Clean_Up Function to work 

In [31]:
df5 =df4.fillna("Hello")

In [32]:
#Analyzing for accuracy 
df5.head(5)

,id,text
0,Hello,In this report we take an inside look at Event...
1,Hello,News and articles about CORONAVIRUS are everyw...
2,Hello,"First of all, Wuhan is a place and not a race,..."
3,Hello,ER Doctors urge a reopening: Lockdown creating...
4,Hello,Governor Andrew Cuomo is considering a tempora...


In [33]:
len(df5)

72

In [34]:
#Cleaning Dataset ~ Removal of stop words, Tokenizing, and creating corpus 

In [35]:
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim
import os

text = df5[['text']].applymap(text_cleanup)['text']
dictionary = Dictionary(text)
dictionary.filter_extremes(no_below=10, no_above=0.8)
corpora = [dictionary.doc2bow(doc) for doc in text]

In [36]:
len(corpora)

72

In [37]:
#Importing Webhose Deduplicated Dataset 
json_data=open('/Users/batman/Downloads/webhose_Covid_dedup (1).json').readlines()
dedup_subset = []
for line in json_data:
    dedup_subset.append(json.loads(line))
print(len(dedup_subset))

25055


In [38]:
#Converting json file to Pandas Dataframe
df6 = pd.DataFrame(dedup_subset, columns=['id', 'title', 'text', 'url'])

In [39]:
df6.head(2)

,id,title,text,url
0,0,"Chinese envoy warns of ""political virus"" in fi...",Chinese Ambassador to Britain Liu Xiaoming spe...,http://missouri.statenews.net/news/265858618/c...
1,1,China active in global cooperation on R&D of C...,A staff member displays samples of the COVID-1...,http://missouri.statenews.net/news/265858257/c...


In [40]:
len(df6)

25055

In [41]:
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim
import os

text = df6[['text']].applymap(text_cleanup)['text']
dictionary2 = Dictionary(text)
dictionary2.filter_extremes(no_below=10, no_above=0.8)
corpora2 = [dictionary.doc2bow(doc) for doc in text]

In [21]:
len(corpora2)

25055

In [22]:
#Building Similarity Model off Fake News corpora to run each Webhose into in order to check for similarity >= .65 

In [23]:
tf_idf = gensim.models.TfidfModel(corpora)
print(tf_idf)
s = 0
for i in corpora:
    s += len(i)
print(s)

TfidfModel(num_docs=72, num_nnz=8134)
8134


In [25]:
#Creating Gensim Similarity Function to Quantitatively compare Textual Similarities 

In [26]:
sims = gensim.similarities.Similarity('/Users/batman/Desktop',tf_idf[corpora],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

Similarity index with 72 documents in 0 shards (stored under /Users/batman/Desktop)
<class 'gensim.similarities.docsim.Similarity'>


In [27]:
#Limiting Output to ONE Webhose Article for ease of review---  
#If remove [] the output will list similarity scores for 25055 articles

#Output below = one Webhose Article similarity to each of the 72 fake news articles

In [29]:
#Selecting Webhose Article 164.  Output is similarity to each article within the Fake News article dataset (72)
#Can choose any article and will provide the same output.  

In [30]:
index = sims[tf_idf[corpora2[164]]]
for i in range(len(index)):
    print((i + 1, index[i]))

(1, 0.093562014)
(2, 0.0518005)
(3, 0.07816285)
(4, 0.09491711)
(5, 0.10503892)
(6, 0.019243006)
(7, 0.6815319)
(8, 0.22565061)
(9, 0.042050935)
(10, 0.6851101)
(11, 0.01572319)
(12, 0.09602599)
(13, 0.1614952)
(14, 0.034110222)
(15, 0.06373085)
(16, 0.10704161)
(17, 0.050282907)
(18, 0.065846175)
(19, 0.10797555)
(20, 0.09751404)
(21, 0.07650231)
(22, 0.055733494)
(23, 0.049597625)
(24, 0.68660253)
(25, 0.13505265)
(26, 0.036123652)
(27, 0.108018376)
(28, 0.1191143)
(29, 0.2180615)
(30, 0.22252882)
(31, 0.11232691)
(32, 0.05962752)
(33, 0.0)
(34, 0.0)
(35, 0.18836807)
(36, 0.056788467)
(37, 0.017015267)
(38, 0.0474091)
(39, 0.1627324)
(40, 0.0)
(41, 0.0)
(42, 0.06922154)
(43, 0.03791614)
(44, 0.25143647)
(45, 0.034573726)
(46, 0.12347126)
(47, 0.09947924)
(48, 0.0)
(49, 0.09434501)
(50, 0.0554644)
(51, 0.07367891)
(52, 0.03617637)
(53, 0.67961276)
(54, 0.058161665)
(55, 0.05550503)
(56, 0.09735711)
(57, 0.11793693)
(58, 0.034652963)
(59, 0.03934589)
(60, 0.093068905)
(61, 0.081756085)

In [31]:
#Another Webhose Artile (631) ---except only printing when sim score >= .65 (631 obtained from listing below)

k=0
z=0
zz= .65
index = sims[tf_idf[corpora2[631]]]
for i in range(len(index)):
    if index[i] >= zz:
        print((i + 1, index[i]))

(7, 0.76958346)
(10, 0.733504)
(24, 0.7291062)
(53, 0.72178346)


In [32]:
#We are comparing the similarity of each Webhose Deduplicated Article Text to each of the 72 Fake News Article Texts
#The result below is limited to 1 article for ease of review.  

#As you will see, the results are an Array ---listing Webhose Article and Similarity to each of the 72 
#Fake News Articles.  

In [33]:
Copy = []
Total = []
i= 0
x= 0
z= 0.65
while x < 25055:
    web1 = corpora2[i]
    vector = sims[tf_idf[web1]] 
    for k in vector:
        if k >= z:
            #print(x+1, k)
            Copy.append(x)
            Total.append(k)
    x= x+1
    i= i +1 

In [34]:
#Copy contains all Webhose Articles that are Similar to Fake News text (Similarity equals >=.65)
#print(Copy)
#print(Total)

In [35]:
#Observation shows that one Webhose Article often has a Similarity Score of >= .65 to more than one Fake News Article
#For example, Article 164 is similar to Fake News articles 7, 10, 24, and 53

#Creating a for loop to remove redundancy of Webhose Articles already classified as fake news.  

res = [] 
for i in Copy: 
    if i not in res: 
        res.append(i) 

In [36]:
#The total Number of Webhose Articles in the Webhose dataset that are 'similar' to fake news text
len(res)

266

In [37]:
#Performaing manual inspection 
print(df6.iloc[164])

id                                                     274
title    Goal: 30,000-plus masks – Santa Barbara News-P...
text     Santa Barbara Bucket Brigade makes progress du...
url      https://ten15am.org/goal-30000-plus-masks-sant...
Name: 164, dtype: object


In [38]:
#Printing Articles that are similar to fake news text by index 
print(res)

[164, 305, 418, 585, 631, 657, 729, 1357, 1396, 1429, 1700, 1743, 1746, 1750, 2000, 2371, 2462, 2477, 2528, 2586, 2866, 2888, 2906, 2995, 3589, 3652, 3674, 3694, 3747, 3821, 3961, 4043, 4048, 4106, 4203, 4455, 4613, 4675, 4680, 4802, 4930, 4944, 4949, 5094, 5293, 5405, 5449, 5499, 5523, 5778, 5848, 5864, 5943, 5972, 6310, 6312, 6319, 6344, 6439, 6444, 6503, 6541, 6557, 6634, 6849, 6901, 6904, 7074, 7212, 7250, 7411, 7531, 7548, 7740, 7748, 7829, 7949, 7953, 8119, 8144, 8161, 8220, 8402, 8492, 8520, 8535, 8579, 8603, 8635, 8654, 8668, 8683, 8686, 8800, 8810, 8872, 8907, 8995, 9014, 9097, 9159, 9239, 9318, 9333, 9369, 9399, 9512, 9520, 9601, 9649, 9829, 9836, 9838, 9896, 9900, 10017, 10029, 10037, 10098, 10111, 10159, 10247, 10456, 10550, 10560, 10673, 10714, 10797, 10851, 10859, 10895, 10905, 10910, 10938, 11202, 11296, 11355, 11436, 11489, 11664, 11778, 11920, 12150, 12207, 12220, 12313, 12437, 12478, 12532, 12605, 12655, 12741, 12859, 12862, 12869, 12909, 12910, 12987, 13165, 13185, 1

In [39]:
#Dropping the identified fake news articles from our Webhose dataset ~ leaving us with our Webhose Real dataset

In [40]:
df8 = df6.drop(df6.index[[164, 305, 418, 585, 631, 657, 729, 1357, 1396, 1429, 1700, 1743, 1746, 1750, 2000, 
                         2371, 2462, 2477, 2528, 2586, 2866, 2888, 2906, 2995, 3589, 3652, 3674, 3694, 3747, 
                         3821, 3961, 4043, 4048, 4106, 4203, 4455, 4613, 4675, 4680, 4802, 4930, 4944, 4949, 
                         5094, 5293, 5405, 5449, 5499, 5523, 5778, 5848, 5864, 5943, 5972, 6310, 6312, 6319, 
                         6344, 6439, 6444, 6503, 6541, 6557, 6634, 6849, 6901, 6904, 7074, 7212, 7250, 7411, 
                         7531, 7548, 7740, 7748, 7829, 7949, 7953, 8119, 8144, 8161, 8220, 8402, 8492, 8520, 
                         8535, 8579, 8603, 8635, 8654, 8668, 8683, 8686, 8800, 8810, 8872, 8907, 8995, 9014, 
                         9097, 9159, 9239, 9318, 9333, 9369, 9399, 9512, 9520, 9601, 9649, 9829, 9836, 9838, 
                         9896, 9900, 10017, 10029, 10037, 10098, 10111, 10159, 10247, 10456, 10550, 10560, 10673, 
                         10714, 10797, 10851, 10859, 10895, 10905, 10910, 10938, 11202, 11296, 11355, 11436, 11489, 
                         11664, 11778, 11920, 12150, 12207, 12220, 12313, 12437, 12478, 12532, 12605, 12655, 12741, 
                         12859, 12862, 12869, 12909, 12910, 12987, 13165, 13185, 13422, 13444, 13764, 13779, 13897, 
                         13933, 13967, 14326, 14535, 14621, 14777, 14843, 15026, 15075, 15301, 15379, 15915, 16083, 
                         16208, 16297, 16360, 16478, 16589, 16709, 16850, 16909, 16973, 17130, 17320, 17349, 17355, 
                         17581, 17914, 18245, 18411, 18481, 18517, 18551, 18624, 18751, 18793, 18881, 19550, 19638, 
                         19691, 19944, 19981, 19989, 20056, 20379, 20686, 20694, 20731, 20999, 21064, 21096, 21195, 
                         21297, 21298, 21547, 21619, 21687, 21742, 22169, 22178, 22263, 22285, 22324, 22384, 22404, 
                         22472, 22548, 22559, 22692, 22813, 23081, 23112, 23125, 23137, 23165, 23315, 23564, 23630, 
                         23686, 23779, 23885, 23902, 23916, 23921, 24049, 24132, 24245, 24278, 24306, 24350, 24383, 
                         24422, 24470, 24596, 24649, 24705, 24749, 24810, 24891, 25008, 25031]])

In [41]:
len(df8)

24789

In [42]:
#The fruition of our work:  Webhose REAL Dataset with Fake Articles Removed 

In [43]:
#Webhose Real Dataset 
df8.head(100)

,id,title,text,url
0,0,"Chinese envoy warns of ""political virus"" in fi...",Chinese Ambassador to Britain Liu Xiaoming spe...,http://missouri.statenews.net/news/265858618/c...
1,1,China active in global cooperation on R&D of C...,A staff member displays samples of the COVID-1...,http://missouri.statenews.net/news/265858257/c...
2,2,"Blockchain in Education Market Size, by Global...","Blockchain in Education Market Size, by Global...",https://otcpm24.com/2020/07/23/blockchain-in-e...
3,3,Emirates to Cover Virus-Related Medical Costs ...,"Emirates, the largest airline in the Middle Ea...",http://www.naharnet.com/stories/en/273613-emir...
4,4,Portland mayor teargassed as calls mount for T...,"City’s Democratic mayor, Ted Wheeler, teargass...",https://www.theguardian.com/us-news/live/2020/...
...,...,...,...,...
95,155,Aircraft Micro Turbine Engines Market Worth $6...,Aircraft Micro Turbine Engines Market Worth $6...,http://www.sbwire.com/press-releases/aircraft-...
96,156,Huntington Learning Center Offers Academic Hel...,In-Person and Live Online Tutoring and Test Pr...,https://www.newswire.com/news/huntington-learn...
97,157,Sanford Health and TruGenomix announce partner...,"Online Forms Support Local Journalism Now, mor...",https://curated.tncontentexchange.com/states/s...
98,158,Imperva Research Labs Reveals Overall Impact o...,Contact Us Global Distribution Newswire Servic...,https://www.globenewswire.com/news-release/202...


In [44]:
#Creating Webhose Fake dataset via iloc function 

In [45]:
df9 = df6.iloc[[164, 305, 418, 585, 631, 657, 729, 1357, 1396, 1429, 1700, 1743, 1746, 1750, 2000, 
                         2371, 2462, 2477, 2528, 2586, 2866, 2888, 2906, 2995, 3589, 3652, 3674, 3694, 3747, 
                         3821, 3961, 4043, 4048, 4106, 4203, 4455, 4613, 4675, 4680, 4802, 4930, 4944, 4949, 
                         5094, 5293, 5405, 5449, 5499, 5523, 5778, 5848, 5864, 5943, 5972, 6310, 6312, 6319, 
                         6344, 6439, 6444, 6503, 6541, 6557, 6634, 6849, 6901, 6904, 7074, 7212, 7250, 7411, 
                         7531, 7548, 7740, 7748, 7829, 7949, 7953, 8119, 8144, 8161, 8220, 8402, 8492, 8520, 
                         8535, 8579, 8603, 8635, 8654, 8668, 8683, 8686, 8800, 8810, 8872, 8907, 8995, 9014, 
                         9097, 9159, 9239, 9318, 9333, 9369, 9399, 9512, 9520, 9601, 9649, 9829, 9836, 9838, 
                         9896, 9900, 10017, 10029, 10037, 10098, 10111, 10159, 10247, 10456, 10550, 10560, 10673, 
                         10714, 10797, 10851, 10859, 10895, 10905, 10910, 10938, 11202, 11296, 11355, 11436, 11489, 
                         11664, 11778, 11920, 12150, 12207, 12220, 12313, 12437, 12478, 12532, 12605, 12655, 12741, 
                         12859, 12862, 12869, 12909, 12910, 12987, 13165, 13185, 13422, 13444, 13764, 13779, 13897, 
                         13933, 13967, 14326, 14535, 14621, 14777, 14843, 15026, 15075, 15301, 15379, 15915, 16083, 
                         16208, 16297, 16360, 16478, 16589, 16709, 16850, 16909, 16973, 17130, 17320, 17349, 17355, 
                         17581, 17914, 18245, 18411, 18481, 18517, 18551, 18624, 18751, 18793, 18881, 19550, 19638, 
                         19691, 19944, 19981, 19989, 20056, 20379, 20686, 20694, 20731, 20999, 21064, 21096, 21195, 
                         21297, 21298, 21547, 21619, 21687, 21742, 22169, 22178, 22263, 22285, 22324, 22384, 22404, 
                         22472, 22548, 22559, 22692, 22813, 23081, 23112, 23125, 23137, 23165, 23315, 23564, 23630, 
                         23686, 23779, 23885, 23902, 23916, 23921, 24049, 24132, 24245, 24278, 24306, 24350, 24383, 
                         24422, 24470, 24596, 24649, 24705, 24749, 24810, 24891, 25008, 25031]]

In [46]:
#Ensuring proper length of 266
len(df9)

266

In [48]:
#Performing manual inspection of Webhose Fake Dataset.  This information is critical because we can identify proponents of misinformation 
#via the urls (we can red flad these sites)
df9.head(100)

,id,title,text,url
164,274,"Goal: 30,000-plus masks – Santa Barbara News-P...",Santa Barbara Bucket Brigade makes progress du...,https://ten15am.org/goal-30000-plus-masks-sant...
305,505,Do I Have To Wear A Face Mask? What You Need T...,Gov. Eric Holcomb announced Wednesday Indiana ...,http://wvpe.org/post/do-i-have-wear-face-mask-...
418,718,Sweating Under The Face Mask Could Be Causing ...,"Hey, you are not subscribed. Click here to sub...",https://doctor.ndtv.com/skin/sweating-under-th...
585,936,COVID-19: Wrong use of face mask hazardous – L...,"A Medical Laboratory Scientist, Michael Idoko,...",https://dailynigerian.com/covid-19-wrong-use-o...
631,1008,Here Are Ways To Get Your Kids to Wear a Face ...,Confirmed cases of coronavirus disease are sti...,https://www.latinpost.com/articles/146543/2020...
...,...,...,...,...
8872,16714,These KN95 masks are in stock—and on sale—at A...,Yahoo Life Shopping These KN95 masks are in st...,https://www.yahoo.com/lifestyle/kn95-amazon-sa...
8907,16839,MASK OF FUTURE,http://feedproxy.google.com/~r/DrudgeReportFee...,https://clarion.causeaction.com/2020/07/22/mas...
8995,17163,Trump now says 'many people say that it is Pat...,President Donald Trump dramatically shifted hi...,https://www.businessinsider.com/8-times-trump-...
9014,17246,Statewide mask mandate issued by Gov. Walz,Wearing a face mask while indoors is now a req...,https://www.swnewsmedia.com/chaska_herald/stat...


In [139]:
#Isolating the Webhose Fake URLs ---which we believe is the most useful information.  These sites can labeled as proponents of 
#misinformation, i.e. FAKE NEWS
df10=df9['url']

In [137]:
len(df10)

266

In [157]:
#Visual Inspection of the Fake News URLs
df10.head(25)

164     https://ten15am.org/goal-30000-plus-masks-sant...
305     http://wvpe.org/post/do-i-have-wear-face-mask-...
418     https://doctor.ndtv.com/skin/sweating-under-th...
585     https://dailynigerian.com/covid-19-wrong-use-o...
631     https://www.latinpost.com/articles/146543/2020...
657     https://themazatlanpost.com/2020/07/23/amlo-do...
729     https://www.wlwt.com/article/dewine-more-than-...
1357    https://www.southcoastregister.com.au/story/68...
1396    https://www.tomsguide.com/news/imasc-breakthro...
1429    https://news.sky.com/story/coronavirus-what-sh...
1700            http://www.teo-education.com/teo/?p=55594
1743    https://www.bignewsnetwork.com/news/265866540/...
1746            http://www.teo-education.com/teo/?p=55592
1750    https://www.ladbible.com/news/uk-face-masks-co...
2000    https://www.sciencealert.com/new-prototype-n95...
2371    https://www.realclearhealth.com/2020/07/23/gov...
2462            http://www.teo-education.com/teo/?p=55579
2477    https:

#Entity Analysis:  of Webhose Real + Webhose Fake + Original Fake News Datasets

In [107]:
len(df8)

24789

In [108]:
#Extracting text from Webhose Real dataset / datafram 

news_df=df8['text']

In [109]:
len(news_df)

24789

In [110]:
#Inspecting the df to ensure proper text extraction 
news_df.head(20)

0     Chinese Ambassador to Britain Liu Xiaoming spe...
1     A staff member displays samples of the COVID-1...
2     Blockchain in Education Market Size, by Global...
3     Emirates, the largest airline in the Middle Ea...
4     City’s Democratic mayor, Ted Wheeler, teargass...
5     Atlanta-Based Citiri Receives Growth Investmen...
6     2020 Catholic Immigrant Integration Initiative...
7     Picture Supply : AP\nFILE The World Well being...
8     /PRNewswire/ -- Shreis Scalene Therapeutics LL...
9     Zynex Schedules 2020 Second Quarter Earnings N...
10    DUBLIN–( BUSINESS WIRE )–The “Europe Data Prot...
11    SINGAPORE – Mr Khaw Boon Wan’s dengue diagnosi...
12    160 Persons Test Positive for Coronavirus in U...
13    /PRNewswire/ -- The video inspection equipment...
14    Five countries have been identified as respons...
15    6 Share\nAnurag Kashyap said that while Karan ...
16    Instead of waiting for instructions from autho...
17    Last week, a Black Lives Matter business o

In [111]:
news_df.tail(20)

25035    After falling for the first time in 10 weeks, ...
25036    I am a lifelong resident of Orofino and a Regi...
25037    News and Notes\nby Venturans for Responsible &...
25038    By pixelheadphoto digitalskillet\nOPINION\nLik...
25039    Ventura Art Scene A unique old Italian techniq...
25040    Brazil Reports Record Cases; Texas Deaths Hit ...
25041    Knowing where to go to get the care you need c...
25042    Author response: Neurologic manifestations in ...
25043    By Beth Lawrence The Jackson County Board of C...
25044    The world today is facing unprecedented challe...
25045    Dear Business Owners, Community Members, Paren...
25046    In about two weeks millions of Americans could...
25047    The coronavirus crisis has created a critical ...
25048    Researchers have conducted one of the first st...
25049    Who We’re Supporting: American Red Cross - CA ...
25050    autoimmune diseases We have read the recent re...
25051    Story collection: Chronicling the impact of th.

In [112]:
#Importing Spacy libraries for named entities recognition
!pip install spacy 
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.util import minibatch, compounding
from pathlib import Path
import random

We will be using SpaCy’s named entity recognition to search for the following entity types:  Person and Organization

In [113]:
#Get the collection of entities from our Webhose Real dataset

doc = nlp(str(news_df))
print([(word, word.ent_type_) for word in doc if word.ent_type_])

[(Chinese, 'NORP'), (Britain, 'PERSON'), (Liu, 'PERSON'), (Xiaoming, 'PERSON'), (1, 'CARDINAL'), (2, 'CARDINAL'), (3, 'CARDINAL'), (the, 'LOC'), (Middle, 'LOC'), (Ea, 'LOC'), (4, 'CARDINAL'), (Democratic, 'NORP'), (Ted, 'PERSON'), (Wheeler, 'PERSON'), (25050, 'DATE'), (25051, 'CARDINAL'), (25052, 'CARDINAL'), (25053, 'CARDINAL'), (22, 'CARDINAL'), (South, 'LOC'), (Asia, 'LOC'), ('s, 'LOC'), (24789, 'CARDINAL')]


In [114]:
#Our collection of entities in the articles are represented with the below unique labels:
labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter({'NORP': 2, 'PERSON': 2, 'CARDINAL': 9, 'LOC': 2, 'DATE': 1})

In [134]:
# This version shows the most frequent Tokens in the Webhose Real dataset without naming the type of entities.
items = [x.text for x in doc.ents]
Counter(items).most_common()

[('Chinese', 1),
 ('Britain Liu Xiaoming', 1),
 ('1', 1),
 ('2', 1),
 ('3', 1),
 ('the Middle Ea', 1),
 ('4', 1),
 ('Democratic', 1),
 ('Ted Wheeler', 1),
 ('25050', 1),
 ('25051', 1),
 ('25052', 1),
 ('25053', 1),
 ('22', 1),
 ("South Asia's", 1),
 ('24789', 1)]

In [116]:
#2nd Method of determining the entities 
new_labels=[(i, i.label_, i.label) for i in doc.ents] 
Counter(new_labels)

Counter({(Chinese, 'NORP', 381): 1,
         (Britain Liu Xiaoming, 'PERSON', 380): 1,
         (1, 'CARDINAL', 397): 1,
         (2, 'CARDINAL', 397): 1,
         (3, 'CARDINAL', 397): 1,
         (the Middle Ea, 'LOC', 385): 1,
         (4, 'CARDINAL', 397): 1,
         (Democratic, 'NORP', 381): 1,
         (Ted Wheeler, 'PERSON', 380): 1,
         (25050, 'DATE', 391): 1,
         (25051, 'CARDINAL', 397): 1,
         (25052, 'CARDINAL', 397): 1,
         (25053, 'CARDINAL', 397): 1,
         (22, 'CARDINAL', 397): 1,
         (South Asia's, 'LOC', 385): 1,
         (24789, 'CARDINAL', 397): 1})

In [117]:
#The following are the most frequent tokens
Counter(new_labels).most_common()

[((Chinese, 'NORP', 381), 1),
 ((Britain Liu Xiaoming, 'PERSON', 380), 1),
 ((1, 'CARDINAL', 397), 1),
 ((2, 'CARDINAL', 397), 1),
 ((3, 'CARDINAL', 397), 1),
 ((the Middle Ea, 'LOC', 385), 1),
 ((4, 'CARDINAL', 397), 1),
 ((Democratic, 'NORP', 381), 1),
 ((Ted Wheeler, 'PERSON', 380), 1),
 ((25050, 'DATE', 391), 1),
 ((25051, 'CARDINAL', 397), 1),
 ((25052, 'CARDINAL', 397), 1),
 ((25053, 'CARDINAL', 397), 1),
 ((22, 'CARDINAL', 397), 1),
 ((South Asia's, 'LOC', 385), 1),
 ((24789, 'CARDINAL', 397), 1)]

 Let's analyze our Webhose Fake entities and compare the findings

In [118]:
df9.head(10)

,id,title,text,url
164,274,"Goal: 30,000-plus masks – Santa Barbara News-P...",Santa Barbara Bucket Brigade makes progress du...,https://ten15am.org/goal-30000-plus-masks-sant...
305,505,Do I Have To Wear A Face Mask? What You Need T...,Gov. Eric Holcomb announced Wednesday Indiana ...,http://wvpe.org/post/do-i-have-wear-face-mask-...
418,718,Sweating Under The Face Mask Could Be Causing ...,"Hey, you are not subscribed. Click here to sub...",https://doctor.ndtv.com/skin/sweating-under-th...
585,936,COVID-19: Wrong use of face mask hazardous – L...,"A Medical Laboratory Scientist, Michael Idoko,...",https://dailynigerian.com/covid-19-wrong-use-o...
631,1008,Here Are Ways To Get Your Kids to Wear a Face ...,Confirmed cases of coronavirus disease are sti...,https://www.latinpost.com/articles/146543/2020...
657,1048,AMLO downplays the importance of wearing face ...,Home National AMLO downplays the importance of...,https://themazatlanpost.com/2020/07/23/amlo-do...
729,1223,DeWine: More than 25 Ohio students test positi...,DeWine: More than 25 Ohio students test positi...,https://www.wlwt.com/article/dewine-more-than-...
1357,2366,Face masks are a sign of respect for others as...,Latest News Editorial\nA little over six month...,https://www.southcoastregister.com.au/story/68...
1396,2431,iMASC breakthrough rivals N95 masks — and it’s...,Researchers at MIT and Brigham and Women’s Hos...,https://www.tomsguide.com/news/imasc-breakthro...
1429,2487,Coronavirus: What shops do I have to wear a fa...,Face coverings will be compulsory in shops in ...,https://news.sky.com/story/coronavirus-what-sh...


In [119]:
#Checking the size of the data. We have 72 articles
len(df9)

266

In [120]:
#Extracting the text from Webhose Fake articles 
Fake_articles=df9['text']
Fake_articles.head(20)

164     Santa Barbara Bucket Brigade makes progress du...
305     Gov. Eric Holcomb announced Wednesday Indiana ...
418     Hey, you are not subscribed. Click here to sub...
585     A Medical Laboratory Scientist, Michael Idoko,...
631     Confirmed cases of coronavirus disease are sti...
657     Home National AMLO downplays the importance of...
729     DeWine: More than 25 Ohio students test positi...
1357    Latest News Editorial\nA little over six month...
1396    Researchers at MIT and Brigham and Women’s Hos...
1429    Face coverings will be compulsory in shops in ...
1700    Covid-19: Mask-wearing mandatory from Aug 1 Co...
1743    Man who helped U.S. assassinate Qasem Soleiman...
1746    Covid-19: Mask-wearing mandatory from Aug 1 » ...
1750    Face Masks Compulsory In English Coffee Shops ...
2000    Not every mask is equal. As countries around t...
2371    Jacob Sullum July 23, 2020 (Michigan Office of...
2462    NST Leader: Make face masks mandatory The gove...
2477    Starti

In [121]:
#Get the collection of the entities from our Webhose Fake dataset 

news = nlp(str(Fake_articles))
print([(word, word.ent_type_) for word in news if word.ent_type_])

[(164, 'CARDINAL'), (Santa, 'ORG'), (Barbara, 'ORG'), (Bucket, 'ORG'), (Brigade, 'ORG'), (305, 'CARDINAL'), (Eric, 'PERSON'), (Holcomb, 'PERSON'), (Wednesday, 'DATE'), (Indiana, 'GPE'), (418, 'CARDINAL'), (585, 'CARDINAL'), (Michael, 'PERSON'), (Idoko, 'PERSON'), (631, 'CARDINAL'), (Wednesday, 'DATE'), (,, 'DATE'), (July, 'DATE'), (22, 'DATE'), (,, 'DATE'), (2020, 'DATE'), (24810, 'DATE'), (Nathan, 'ORG'), (Frey, 'ORG'), (|, 'ORG'), (Jul, 'ORG'), (20, 'ORG'), (Holcomb, 'PERSON'), (25031, 'CARDINAL'), (Length, 'PERSON'), (266, 'CARDINAL')]


In [122]:
#Checking the size of entities in our corpus
len(news)

143

In [123]:
#Our collection of entities in the Webhose Fake articles are represented by 5 unique labels:
labels_news = [x.label_ for x in news.ents]
Counter(labels_news)

Counter({'CARDINAL': 7, 'ORG': 2, 'PERSON': 4, 'DATE': 3, 'GPE': 1})

In [124]:
#2nd Method: determine the entities of each token with the description of the type 
labels_news2=[(i, i.label_, i.label) for i in news.ents] 
Counter(labels_news2)

Counter({(164, 'CARDINAL', 397): 1,
         (Santa Barbara Bucket Brigade, 'ORG', 383): 1,
         (305, 'CARDINAL', 397): 1,
         (Eric Holcomb, 'PERSON', 380): 1,
         (Wednesday, 'DATE', 391): 1,
         (Indiana, 'GPE', 384): 1,
         (418, 'CARDINAL', 397): 1,
         (585, 'CARDINAL', 397): 1,
         (Michael Idoko, 'PERSON', 380): 1,
         (631, 'CARDINAL', 397): 1,
         (Wednesday, July 22, 2020, 'DATE', 391): 1,
         (24810, 'DATE', 391): 1,
         (Nathan Frey | Jul 20, 'ORG', 383): 1,
         (Holcomb, 'PERSON', 380): 1,
         (25031, 'CARDINAL', 397): 1,
         (Length, 'PERSON', 380): 1,
         (266, 'CARDINAL', 397): 1})

In [125]:
#The following are the most frequent tokens in the Webhose Fake dataset 
Counter(labels_news2).most_common(50)

[((164, 'CARDINAL', 397), 1),
 ((Santa Barbara Bucket Brigade, 'ORG', 383), 1),
 ((305, 'CARDINAL', 397), 1),
 ((Eric Holcomb, 'PERSON', 380), 1),
 ((Wednesday, 'DATE', 391), 1),
 ((Indiana, 'GPE', 384), 1),
 ((418, 'CARDINAL', 397), 1),
 ((585, 'CARDINAL', 397), 1),
 ((Michael Idoko, 'PERSON', 380), 1),
 ((631, 'CARDINAL', 397), 1),
 ((Wednesday, July 22, 2020, 'DATE', 391), 1),
 ((24810, 'DATE', 391), 1),
 ((Nathan Frey | Jul 20, 'ORG', 383), 1),
 ((Holcomb, 'PERSON', 380), 1),
 ((25031, 'CARDINAL', 397), 1),
 ((Length, 'PERSON', 380), 1),
 ((266, 'CARDINAL', 397), 1)]

Analyzing the original Fake News dataset itself ---obtained via NewsGuardATech.com and not derived from our Webhose Dataset

In [126]:
dfOrigFake = pd.read_csv('/Users/batman/Desktop/fake_news.csv')

In [127]:
dfOrigFake.head(5)

,ur1,title,url2,text
0,ActivistPost.com,Did Bill Gates and world economic forum predic...,https://www.nexusnewsfeed.com/article/human-ri...,In this report we take an inside look at Event...
1,AgeOfAutism.com,Coronavirus Can Be Caused By Viral Interferenc...,https://www.ageofautism.com/2020/01/coronaviru...,News and articles about CORONAVIRUS are everyw...
2,AmericanThinker.com,The Wuhan Virus Escaped from a Chinese Lab,https://www.americanthinker.com/articles/2020/...,"First of all, Wuhan is a place and not a race,..."
3,BB4SP.com,ER Doctors urge a reopening: Lockdown creating...,https://bb4sp.com/e-r-doctors-urge-a-reopening...,ER Doctors urge a reopening: Lockdown creating...
4,BuffaloChronicle.com,Cuomo considers banning cigarette sales for si...,https://buffalochronicle.com/2020/03/27/cuomo-...,Governor Andrew Cuomo is considering a tempora...


In [128]:
#Extracting only text
text=dfOrigFake['text']
text.head(20)

0     In this report we take an inside look at Event...
1     News and articles about CORONAVIRUS are everyw...
2     First of all, Wuhan is a place and not a race,...
3     ER Doctors urge a reopening: Lockdown creating...
4     Governor Andrew Cuomo is considering a tempora...
5     Last night the first confirmed case of the cor...
6     There has been a shifting of positions on the ...
7     Mary and I live in a tiny town in West Virgini...
8     The Facts:A study published in the journal Vac...
9     New information about face masks has emerged, ...
10    Last weekend Dr. Vladimir Zelenko from New Yor...
11    The “Blue Plague” is an intentional effort by ...
12    With close to 200,000 cases of coronavirus acr...
13    (Natural News) Every virology lab in the world...
14    (Natural News) Two days ago, a paper published...
15    Social distancing is being enforced across the...
16    The medical community strongly advises the pop...
17    Throughout the state’s history (save the l

In [129]:
#Get the collection of the entities from our data 

news3 = nlp(str(text))
print([(word, word.ent_type_) for word in news3 if word.ent_type_])

[(1, 'CARDINAL'), (CORONAVIRUS, 'ORG'), (2, 'CARDINAL'), (First, 'ORDINAL'), (Wuhan, 'GPE'), (3, 'CARDINAL'), (4, 'CARDINAL'), (Andrew, 'PERSON'), (Cuomo, 'PERSON'), (69, 'CARDINAL'), (70, 'CARDINAL'), (71, 'CARDINAL'), (Belgium, 'GPE'), (Maggie, 'PERSON'), (de, 'PERSON'), (Block, 'PERSON'), (72, 'CARDINAL'), (July, 'DATE'), (2nd, 'DATE'), (Texas, 'GPE'), (Greg, 'PERSON'), (Abbott, 'PERSON'), (73, 'CARDINAL'), (2019, 'DATE'), (Length, 'PERSON'), (74, 'CARDINAL')]


In [130]:
#Checking the size of entities in our corpus
len(news3)

144

In [131]:
#Our collection of entities in the Original Fake News dataset are represented by these labels:
labels_news3 = [x.label_ for x in news3.ents]
Counter(labels_news3)

Counter({'CARDINAL': 10,
         'ORG': 1,
         'ORDINAL': 1,
         'GPE': 3,
         'PERSON': 4,
         'DATE': 2})

In [132]:
#2nd Method: determine the entities of each token with the description of the type 
labels_fn=[(i, i.label_, i.label) for i in news3.ents] 
Counter(labels_fn)

Counter({(1, 'CARDINAL', 397): 1,
         (CORONAVIRUS, 'ORG', 383): 1,
         (2, 'CARDINAL', 397): 1,
         (First, 'ORDINAL', 396): 1,
         (Wuhan, 'GPE', 384): 1,
         (3, 'CARDINAL', 397): 1,
         (4, 'CARDINAL', 397): 1,
         (Andrew Cuomo, 'PERSON', 380): 1,
         (69, 'CARDINAL', 397): 1,
         (70, 'CARDINAL', 397): 1,
         (71, 'CARDINAL', 397): 1,
         (Belgium, 'GPE', 384): 1,
         (Maggie de Block, 'PERSON', 380): 1,
         (72, 'CARDINAL', 397): 1,
         (July 2nd, 'DATE', 391): 1,
         (Texas, 'GPE', 384): 1,
         (Greg Abbott, 'PERSON', 380): 1,
         (73, 'CARDINAL', 397): 1,
         (2019, 'DATE', 391): 1,
         (Length, 'PERSON', 380): 1,
         (74, 'CARDINAL', 397): 1})

In [133]:
#The following are the most frequent tokens in the Original Fake News Dataset 
Counter(labels_fn).most_common()

[((1, 'CARDINAL', 397), 1),
 ((CORONAVIRUS, 'ORG', 383), 1),
 ((2, 'CARDINAL', 397), 1),
 ((First, 'ORDINAL', 396), 1),
 ((Wuhan, 'GPE', 384), 1),
 ((3, 'CARDINAL', 397), 1),
 ((4, 'CARDINAL', 397), 1),
 ((Andrew Cuomo, 'PERSON', 380), 1),
 ((69, 'CARDINAL', 397), 1),
 ((70, 'CARDINAL', 397), 1),
 ((71, 'CARDINAL', 397), 1),
 ((Belgium, 'GPE', 384), 1),
 ((Maggie de Block, 'PERSON', 380), 1),
 ((72, 'CARDINAL', 397), 1),
 ((July 2nd, 'DATE', 391), 1),
 ((Texas, 'GPE', 384), 1),
 ((Greg Abbott, 'PERSON', 380), 1),
 ((73, 'CARDINAL', 397), 1),
 ((2019, 'DATE', 391), 1),
 ((Length, 'PERSON', 380), 1),
 ((74, 'CARDINAL', 397), 1)]

#Entity Analysis:  SpaCY's NER lacks accuracy without Training Data.  Subsequent steps would involve the developement of a training model to improve accuracy for Organizations and People.     

In [ ]:
End of Semester NLP Project. 

In [1]:
pwd

'/Users/batman/Desktop'

In [2]:
import pandas as pd 
import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors
from operator import itemgetter, attrgetter

In [3]:
model_path = '/Users/batman/Downloads/'

In [4]:
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin.gz', True)
#model_word2vec = load_wordvec_model('Word2Vec', 'webhose_skipgram_300.bin', True)

Loading Word2Vec model...
Finished loading Word2Vec model...


In [5]:
# BUILDING MY OWN TAXONOMY BASED ON LDA and MANUAL DATA EXPLORATION

In [ ]:
topic_taxonomy = {
    "Origination":
    {
        "Government":  "China USA Russia Wuhan North Korea UK",
        "Biological":  "bat cat animal dog corona slakkl kk;",
        "Developed": "employee China USA Russia lab university Harvard"
    },
    "Transmission":
    {

        "Network Security": "unauthorized access intrusion misuse denial",
        "Cloud Security":  "data applications services infrastructure",
        "Backup and Recovery":  "customer duplicate files folders cloud"
    },
    "Testing":
    {
        "Shares":    "company quarter ratings news",                                                                   .,l'p
        =
        "Dividends": "average research target report",
        "Price": "purchase ratio valued earnings"
    },
    "Health":
    {
        "Tech Companies":  "Amazon Microsoft Apple Google AWS",
        "Law Enforcement": "digital identification criminal software solution bias",
        "Tech Bias": "race gender ethnicity reliability map features prejudice population demographics",
    },
}